In [99]:
import json
import math
import operator
from collections import OrderedDict
from glob import glob

import pandas as pd

## Transform value maps from table to JSON

1. Export ODS as CSV
2. Remove redundant rows in text editor

In [98]:
df = pd.read_csv('docs/value_maps.csv', header=0)
for col in df.columns.values:
    if col.startswith('Unnamed'):
        del df[col]


def from_fractional(x):
    if '/' in x:
        num, den = x.split('/')
        return float(num) / float(den)
    else:
        return float(x)


def clean_EFXthC(x):
    if x == 'off':
        return float(999)
    else:
        return float(x)


value_maps_json = {}
for col in df.columns.values:
    if col == 'CLA000':
        d = (df[col] * 0.01).tolist()
    elif col == 'OSCprt':
        d = [from_fractional(x) for x in df[col]]
    elif col == 'EFXthC':
        d = [clean_EFXthC(x) for x in df[col]]
    else:
        d = [x for x in df[col]]
    # weird hack to leave only 3 decimals
    value_maps_json[col] = [float(f'{round(x, 3):.3f}') for x in d if not math.isnan(x)]

with open('value_maps.json', 'w') as f:
    json.dump(value_maps_json, f, indent=True)

## Convert old mappings to JSON

In [ ]:
mod_map = OrderedDict()
with open('ModID2Name.txt', 'r') as f:
    for line in f:
        t = line.split()
        if len(t) == 1:
            mod_map[t[0]] = 'Unknown'
        elif len(t) == 2:
            mod_map[t[0]] = t[1]
        else:
            raise RuntimeError(t)
with open('mod_id.json', 'w') as f:
    json.dump(mod_map, f)

In [ ]:
mod_id_connections = OrderedDict()
for fn in glob('IO/*.txt'):
    id = int(fn.replace('IO/', '').replace('.txt', ''))
    data = {'inputs': [], 'outputs': []}
    with open(fn, 'r') as f:
        for line in f:
            if line == '':
                continue
            k, v = line.split()
            if k == 'I':
                data['inputs'].append(v)
            elif k == 'O':
                data['outputs'].append(v)
    mod_id_connections[id] = data
mod_id_connections = OrderedDict(sorted(mod_id_connections.items(), key=operator.itemgetter(0)))
with open('mod_id_connections.json', 'w') as f:
    json.dump(mod_id_connections, f)